
# 🧹 Limpieza de Datasets — Customers & Orders

En este notebook se realiza el proceso de **limpieza y preparación** de los datasets `customers_origen.xlsx` y `orders_origen.xlsx` con el objetivo de crear un **dataset unificado y limpio** para su posterior análisis exploratorio.

---
## 1️⃣ Carga de Datos
Se cargan los archivos desde la carpeta de origen y se muestra una vista previa de su contenido.


In [1]:

import pandas as pd
from IPython.display import display

# Definir rutas
path_customers = r"C:\Users\HUGO\Desktop\Data Analyst\xx_PROYECTO_FINAL\Proyecto_Final\data\archivos_origen\customers_origen.xlsx"
path_orders = r"C:\Users\HUGO\Desktop\Data Analyst\xx_PROYECTO_FINAL\Proyecto_Final\data\archivos_origen\orders_origen.xlsx"

# Cargar datasets
df_customers = pd.read_excel(path_customers)
df_orders = pd.read_excel(path_orders)

print("✅ Archivos cargados correctamente.\n")
print("Dimensiones df_customers:", df_customers.shape)
print("Dimensiones df_orders:", df_orders.shape)

print("\nVista previa df_customers:")
display(df_customers.head(3))
print("\nVista previa df_orders:")
display(df_orders.head(3))


✅ Archivos cargados correctamente.

Dimensiones df_customers: (5000, 12)
Dimensiones df_orders: (55000, 20)

Vista previa df_customers:


,customer_id,first_name,last_name,email,signup_date,gender,birth_year,loyalty_tier,preferred_language,avg_order_value,num_orders,segment
0,CUST_00001,María,Sánchez,user1@example.com,2022-09-22 07:23:44.291,M,1979,Bronze,es,74.47,1,Consumer
1,CUST_00002,Carmen,López,user2@example.com,2022-09-28 15:12:24.261,F,1957,Silver,es,229.48,6,Consumer
2,CUST_00003,Diego,Ruiz,user3@example.com,2017-11-13 23:33:12.565,Other,1975,Bronze,es,72.89,8,Consumer



Vista previa df_orders:


,order_id,order_date,customer_id,product_id,category,brand,quantity,unit_price,discount_pct,total_price,payment_method,shipping_method,order_status,city,state,country,zip_code,sales_region,fulfillment_center,returned_flag
0,ORD_0000001,2023-01-03 00:07:51.619,CUST_04882,PROD_0595,Electronics,BrandB,4,272.16,0.15,925.34,card,standard,delivered,Málaga,Basque Country,Spain,50597,Center,FC3,0
1,ORD_0000002,2020-04-21 16:54:38.557,CUST_04860,PROD_0289,Electronics,BrandX,1,335.09,0.00,335.09,card,standard,delivered,Bilbao,Valencia,Spain,10914,Center,FC2,0
2,ORD_0000003,2021-06-16 12:59:40.818,CUST_02539,PROD_0236,Sports,BrandA,1,361.43,0.10,325.29,cash_on_delivery,standard,delivered,Sevilla,Valencia,Spain,18910,East,FC4,0



---
## 2️⃣ Limpieza Global
En este paso se aplican transformaciones comunes a ambos datasets:
- Normalización de nombres de columnas.
- Eliminación de espacios.
- Conversión de fechas.
- Eliminación de duplicados.
- Revisión de valores nulos.


In [2]:
# ====================================
# Limpieza global (versión paso a paso)
# ====================================


# --- 1) Normalizar nombres de columnas ---
print(">> 1) Normalización de nombres de columnas")
old_cust_cols = df_customers.columns.tolist()
old_orders_cols = df_orders.columns.tolist()

df_customers.columns = df_customers.columns.str.strip().str.lower().str.replace(" ", "_", regex=False)
df_orders.columns    = df_orders.columns.str.strip().str.lower().str.replace(" ", "_", regex=False)

print(" - Ejemplo columnas customers (antes -> ahora):")
print(old_cust_cols[:10], "->", df_customers.columns.tolist()[:10])
print(" - Ejemplo columnas orders (antes -> ahora):")
print(old_orders_cols[:10], "->", df_orders.columns.tolist()[:10])
print("")

# --- 2) Eliminar espacios en cadenas (solo columnas object/string) ---
print(">> 2) Eliminación de espacios en columnas de texto (solo columnas tipo object/string)")

obj_cols_c = df_customers.select_dtypes(include=["object", "string"]).columns.tolist()
obj_cols_o = df_orders.select_dtypes(include=["object", "string"]).columns.tolist()

print(" - Columns text (customers):", obj_cols_c)
print(" - Columns text (orders):", obj_cols_o)

# Aplicar strip() de forma segura por columna (más rápido y seguro que applymap sobre todo el DF)
for col in obj_cols_c:
    df_customers[col] = df_customers[col].astype("string").str.strip()

for col in obj_cols_o:
    df_orders[col] = df_orders[col].astype("string").str.strip()

print(" - Muestra (customers) columnas text, filas 0..2:")
display(df_customers[obj_cols_c].head(3))
print(" - Muestra (orders) columnas text, filas 0..2:")
display(df_orders[obj_cols_o].head(3))
print("")

# --- 3) Conversión de fechas y verificación ---
print(">> 3) Conversión de columnas de fecha y verificación")

if "signup_date" in df_customers.columns:
    n_before = df_customers["signup_date"].isna().sum()
    df_customers["signup_date"] = pd.to_datetime(df_customers["signup_date"], errors="coerce")
    n_after = df_customers["signup_date"].isna().sum()
    print(f" - signup_date: nulos antes={n_before}, después={n_after}")
    print(" - signup_date min/max:", df_customers["signup_date"].min(), df_customers["signup_date"].max())

if "order_date" in df_orders.columns:
    n_before = df_orders["order_date"].isna().sum()
    df_orders["order_date"] = pd.to_datetime(df_orders["order_date"], errors="coerce")
    n_after = df_orders["order_date"].isna().sum()
    print(f" - order_date: nulos antes={n_before}, después={n_after}")
    print(" - order_date min/max:", df_orders["order_date"].min(), df_orders["order_date"].max())
print("")

# --- 4) Eliminar duplicados (con conteo) ---
print(">> 4) Eliminación de duplicados")
n_before = len(df_customers)
df_customers.drop_duplicates(inplace=True)
n_after = len(df_customers)
print(f" - customers: {n_before - n_after} duplicados eliminados (antes {n_before}, después {n_after})")

n_before = len(df_orders)
df_orders.drop_duplicates(inplace=True)
n_after = len(df_orders)
print(f" - orders: {n_before - n_after} duplicados eliminados (antes {n_before}, después {n_after})")
print("")

# --- 5) Comprobación rápida de claves y tipos ---
print(">> 5) Comprobaciones rápidas")
if "customer_id" in df_customers.columns:
    print(" - Unicidad customer_id (customers):", df_customers["customer_id"].duplicated().sum(), "duplicados")
if "order_id" in df_orders.columns:
    print(" - Unicidad order_id (orders):", df_orders["order_id"].duplicated().sum(), "duplicados")

print(" - Tipos (customers):")
print(df_customers.dtypes)
print(" - Tipos (orders):")
print(df_orders.dtypes)
print("")

# --- 6) Resumen de valores nulos por columna ---
print(">> 6) Valores nulos por columna (top 10 más afectados)")
print(" - df_customers (top 10 columnas con más nulos):")
print(df_customers.isna().sum().sort_values(ascending=False).head(10))
print(" - df_orders (top 10 columnas con más nulos):")
print(df_orders.isna().sum().sort_values(ascending=False).head(10))
print("")

print("✅ Limpieza global (paso a paso) completada.")


>> 1) Normalización de nombres de columnas
 - Ejemplo columnas customers (antes -> ahora):
['customer_id', 'first_name', 'last_name', 'email', 'signup_date', 'gender', 'birth_year', 'loyalty_tier', 'preferred_language', 'avg_order_value'] -> ['customer_id', 'first_name', 'last_name', 'email', 'signup_date', 'gender', 'birth_year', 'loyalty_tier', 'preferred_language', 'avg_order_value']
 - Ejemplo columnas orders (antes -> ahora):
['order_id', 'order_date', 'customer_id', 'product_id', 'category', 'brand', 'quantity', 'unit_price', 'discount_pct', 'total_price'] -> ['order_id', 'order_date', 'customer_id', 'product_id', 'category', 'brand', 'quantity', 'unit_price', 'discount_pct', 'total_price']

>> 2) Eliminación de espacios en columnas de texto (solo columnas tipo object/string)
 - Columns text (customers): ['customer_id', 'first_name', 'last_name', 'email', 'gender', 'loyalty_tier', 'preferred_language', 'segment']
 - Columns text (orders): ['order_id', 'customer_id', 'product_id',

,customer_id,first_name,last_name,email,gender,loyalty_tier,preferred_language,segment
0,CUST_00001,María,Sánchez,user1@example.com,M,Bronze,es,Consumer
1,CUST_00002,Carmen,López,user2@example.com,F,Silver,es,Consumer
2,CUST_00003,Diego,Ruiz,user3@example.com,Other,Bronze,es,Consumer


 - Muestra (orders) columnas text, filas 0..2:


,order_id,customer_id,product_id,category,brand,payment_method,shipping_method,order_status,city,state,country,sales_region,fulfillment_center
0,ORD_0000001,CUST_04882,PROD_0595,Electronics,BrandB,card,standard,delivered,Málaga,Basque Country,Spain,Center,FC3
1,ORD_0000002,CUST_04860,PROD_0289,Electronics,BrandX,card,standard,delivered,Bilbao,Valencia,Spain,Center,FC2
2,ORD_0000003,CUST_02539,PROD_0236,Sports,BrandA,cash_on_delivery,standard,delivered,Sevilla,Valencia,Spain,East,FC4



>> 3) Conversión de columnas de fecha y verificación
 - signup_date: nulos antes=0, después=0
 - signup_date min/max: 2015-01-01 23:28:26.862000 2024-12-28 13:09:15.624000
 - order_date: nulos antes=0, después=0
 - order_date min/max: 2019-01-01 00:15:39.701000 2025-08-31 23:52:03.068000

>> 4) Eliminación de duplicados
 - customers: 0 duplicados eliminados (antes 5000, después 5000)
 - orders: 0 duplicados eliminados (antes 55000, después 55000)

>> 5) Comprobaciones rápidas
 - Unicidad customer_id (customers): 0 duplicados
 - Unicidad order_id (orders): 0 duplicados
 - Tipos (customers):
customer_id           string[python]
first_name            string[python]
last_name             string[python]
email                 string[python]
signup_date           datetime64[ns]
gender                string[python]
birth_year                     int64
loyalty_tier          string[python]
preferred_language    string[python]
avg_order_value              float64
num_orders                     i


---
## 3️⃣ Limpieza Específica por Dataset
Se tratan las particularidades de cada dataset.


In [3]:

# =================================
# Limpieza específica: df_customers
# =================================
print("\n🔹 Limpieza específica de df_customers")
print("\nTipos de datos:")
print(df_customers.dtypes)

print("\nValores únicos en 'gender':", df_customers["gender"].unique())
print("Valores únicos en 'loyalty_tier':", df_customers["loyalty_tier"].unique())
print("Valores únicos en 'segment':", df_customers["segment"].unique())

# =================================
# Limpieza específica: df_orders
# =================================
print("\n🔹 Limpieza específica de df_orders")
print("\nTipos de datos:")
print(df_orders.dtypes)

cols_num = ["quantity", "unit_price", "discount_pct", "total_price"]
for col in cols_num:
    if col in df_orders.columns:
        df_orders[col] = pd.to_numeric(df_orders[col], errors="coerce")

for col in ["payment_method", "shipping_method", "order_status", "returned_flag"]:
    if col in df_orders.columns:
        print(f"\nValores únicos en {col}:", df_orders[col].unique())

print("\n✅ Limpieza específica completada correctamente.")



🔹 Limpieza específica de df_customers

Tipos de datos:
customer_id           string[python]
first_name            string[python]
last_name             string[python]
email                 string[python]
signup_date           datetime64[ns]
gender                string[python]
birth_year                     int64
loyalty_tier          string[python]
preferred_language    string[python]
avg_order_value              float64
num_orders                     int64
segment               string[python]
dtype: object

Valores únicos en 'gender': <StringArray>
['M', 'F', 'Other']
Length: 3, dtype: string
Valores únicos en 'loyalty_tier': <StringArray>
['Bronze', 'Silver', 'Gold', 'Platinum']
Length: 4, dtype: string
Valores únicos en 'segment': <StringArray>
['Consumer', 'Small Business', 'Corporate']
Length: 3, dtype: string

🔹 Limpieza específica de df_orders

Tipos de datos:
order_id              string[python]
order_date            datetime64[ns]
customer_id           string[python]
product_


---
## 4️⃣ Unión de Datasets
Se realiza la unión por `customer_id`.


In [4]:

df_final = pd.merge(df_orders, df_customers, on="customer_id", how="left")
print("✅ Unión completada correctamente.")
print("Dimensiones del DataFrame final:", df_final.shape)
display(df_final.head(5))


✅ Unión completada correctamente.
Dimensiones del DataFrame final: (55000, 31)


,order_id,order_date,customer_id,product_id,category,brand,quantity,unit_price,discount_pct,total_price,...,last_name,email,signup_date,gender,birth_year,loyalty_tier,preferred_language,avg_order_value,num_orders,segment
0,ORD_0000001,2023-01-03 00:07:51.619,CUST_04882,PROD_0595,Electronics,BrandB,4,272.16,0.15,925.34,...,Fernández,user4882@example.com,2019-11-19 09:43:07.056,F,1952,Gold,es,19.29,4,Consumer
1,ORD_0000002,2020-04-21 16:54:38.557,CUST_04860,PROD_0289,Electronics,BrandX,1,335.09,0.00,335.09,...,Hernández,user4860@example.com,2016-04-15 00:54:16.572,M,1946,Silver,pt,28.57,11,Small Business
2,ORD_0000003,2021-06-16 12:59:40.818,CUST_02539,PROD_0236,Sports,BrandA,1,361.43,0.10,325.29,...,Martínez,user2539@example.com,2015-11-23 05:48:11.859,F,1981,Silver,es,38.00,5,Consumer
3,ORD_0000004,2021-12-10 19:33:51.800,CUST_02342,PROD_0181,Books,BrandC,5,272.43,0.20,1089.72,...,Fernández,user2342@example.com,2020-04-13 02:50:25.796,Other,1945,Gold,es,193.97,10,Consumer
4,ORD_0000005,2020-10-14 12:39:56.453,CUST_04751,PROD_0551,Beauty,BrandZ,3,117.11,0.20,281.06,...,Gómez,user4751@example.com,2022-05-21 07:50:04.618,M,1979,Silver,es,79.37,2,Consumer



---
## 5️⃣ Limpieza Post-Unión
Verificación de duplicados, nulos y creación de variables derivadas.


In [5]:

# Verificación general
print("Duplicados totales:", df_final.duplicated().sum())
print("\nValores nulos:")
print(df_final.isna().sum())

# Crear variables derivadas
if "order_date" in df_final.columns:
    df_final["order_year"] = df_final["order_date"].dt.year
    df_final["order_month"] = df_final["order_date"].dt.to_period("M")

print("\n✅ Limpieza post-unión completada correctamente.")


Duplicados totales: 0

Valores nulos:
order_id              0
order_date            0
customer_id           0
product_id            0
category              0
brand                 0
quantity              0
unit_price            0
discount_pct          0
total_price           0
payment_method        0
shipping_method       0
order_status          0
city                  0
state                 0
country               0
zip_code              0
sales_region          0
fulfillment_center    0
returned_flag         0
first_name            0
last_name             0
email                 0
signup_date           0
gender                0
birth_year            0
loyalty_tier          0
preferred_language    0
avg_order_value       0
num_orders            0
segment               0
dtype: int64

✅ Limpieza post-unión completada correctamente.



---
## 6️⃣ Guardado Final
Se exporta el dataset limpio combinado a la carpeta de destino.


In [6]:

output_path = r"C:\Users\HUGO\Desktop\Data Analyst\xx_PROYECTO_FINAL\Proyecto_Final\data\archivos_final\dataset_final.xlsx"
df_final.to_excel(output_path, index=False)
print(f"✅ Archivo final exportado correctamente en: {output_path}")


✅ Archivo final exportado correctamente en: C:\Users\HUGO\Desktop\Data Analyst\xx_PROYECTO_FINAL\Proyecto_Final\data\archivos_final\dataset_final.xlsx



---
# ✅ Resumen del Proceso

Se ha completado la limpieza y unión de los datasets de clientes y pedidos.  
El resultado final es un archivo `.xlsx` listo para análisis exploratorios posteriores.

**Acciones realizadas:**
- Estandarización de columnas y formatos.
- Limpieza de espacios y duplicados.
- Conversión de tipos de datos.
- Validación de categorías clave.
- Unión de `customers` y `orders` mediante `customer_id`.
- Exportación final del dataset limpio.

📂 **Ruta final del archivo:**  
`C:\Users\HUGO\Desktop\Data Analyst\xx_PROYECTO_FINAL\Proyecto_Final\data\archivos_final\dataset_final.xlsx`

---

